In [ ]:
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
matplotlib.rcParams['figure.figsize'] = (12.0, 4.0)
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')

In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)

from braindecode.torch_ext.util import np_to_var

In [ ]:
import os
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')
from autodiag.dataset import DiagnosisSet
import resampy
import numpy as np
from braindecode.datautil.signal_target import SignalAndTarget
max_abs_val = 800
sec_to_cut = 60
duration_recording_mins = 20
n_recordings = 5
sampling_freq = 100
divisor = 10
max_recording_mins = 35
preproc_functions = []
preproc_functions.append(
    lambda data, fs: (data[:, int(sec_to_cut * fs):-int(
        sec_to_cut * fs)], fs))
preproc_functions.append(
    lambda data, fs: (data[:, :int(duration_recording_mins * 60 * fs)], fs))

preproc_functions.append(lambda data, fs:
                         (np.clip(data, -max_abs_val, max_abs_val), fs))


preproc_functions.append(lambda data, fs: (resampy.resample(data, fs,
                                                            sampling_freq,
                                                            axis=1,
                                                            filter='kaiser_fast'),
                                           sampling_freq))

preproc_functions.append(lambda data, fs: (data / divisor, fs))
dataset =DiagnosisSet(n_recordings=n_recordings,
                           max_recording_mins=max_recording_mins,
                           preproc_functions=preproc_functions,
                           train_or_eval='train')

In [ ]:
X,y = dataset.load()
dataset = SignalAndTarget(X,y)

In [ ]:
from braindecode.datautil.iterators import CropsFromTrialsIterator
batch_size = 64
input_time_length = 1200
n_preds_per_input = 600
iterator = CropsFromTrialsIterator(batch_size=batch_size,
                                   input_time_length=input_time_length,
                                   n_preds_per_input=n_preds_per_input)

In [ ]:
for setname, y_class in (('normal', 0), ('abnormal', 1)):
    this_X = [x for x,y in zip(dataset.X, dataset.y) if y == y_class]

    this_y =  dataset.y[dataset.y == y_class]

    assert np.all(this_y == y_class)
    this_set = SignalAndTarget(this_X, this_y)
    batches = list(iterator.get_batches(this_set, shuffle=False))
    batches_arr = np.concatenate(list(zip(*batches))[0], axis=0)
    del batches
    bps = np.abs(np.fft.rfft(batches_arr, axis=2))
    del batches_arr
    median_bp = np.median(bps, axis=(0,3))
    del bps
    np.save("{:s}-bps.npy".format(setname), median_bp)